## Install pyspark and Basic Intro

* https://www.digitalocean.com/community/tutorials/how-to-install-anaconda-on-ubuntu-18-04-quickstart
* http://spark.apache.org/docs/latest/quick-start.html
* https://www.datacamp.com/community/tutorials/apache-spark-python#gs.fMIIqxM

This notebook is based upon [Introduction to Spark with Python, by Jose A. Dianes](https://github.com/jadianes/spark-py-notebooks)

In [1]:
!pip install findspark

Requirement already satisfied: findspark in /home/gong/anaconda3/lib/python3.7/site-packages (1.3.0)

command to start jupyter notebook:
    
    `$ PYSPARK_DRIVER_PYTHON="jupyter" PYSPARK_DRIVER_PYTHON_OPTS="notebook" pyspark`

In [1]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc,col

from time import time

In [5]:
spark = SparkSession\
    .builder\
    .appName("rdd")\
    .getOrCreate()

In [6]:
sc = spark.sparkContext

In [7]:
sc

<SparkContext master=local[*] appName=PySparkShell>

* an RDD is simply a set of Java or Scala objects representing data.

* DataFrame is higher level abstraction built on top of RDD (optimized by spark) `Dataset[Row]`

* Dataset: getting back some type-safety and use of lambda function

## RDD creation

In this notebook we will introduce two different ways of getting data into the basic Spark data structure, the **Resilient Distributed Dataset** or **RDD**. An RDD is a distributed collection of elements. All work in Spark is expressed as either creating new RDDs, transforming existing RDDs, or calling actions on RDDs to compute a result. Spark automatically distributes the data contained in RDDs across your cluster and parallelizes the operations you perform on them.

#### References

The reference book for these and other Spark related topics is *Learning Spark* by Holden Karau, Andy Konwinski, Patrick Wendell, and Matei Zaharia.  

/home/gong/INSIGHT-Bookshelf/Spark_Learning.pdf

The KDD Cup 1999 competition dataset is described in detail [here](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99).

## Getting the data files  

In this notebook we will use the reduced dataset (10 percent) provided for the KDD Cup 1999, containing nearly half million network interactions. The file is provided as a *Gzip* file that we will download locally.  

import urllib

f = urllib.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

## Creating a RDD from a file  

The most common way of creating an RDD is to load it from a file. Notice that Spark's `textFile` can handle compressed files directly.    

In [8]:
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

In [9]:
type(raw_data)

pyspark.rdd.RDD

Now we have our data file loaded into the `raw_data` RDD.

Without getting into Spark *transformations* and *actions*, the most basic thing we can do to check that we got our RDD contents right is to `count()` the number of lines loaded from the file into the RDD.  

In [7]:
raw_data.count()
# 494021

494021

We can also check the first few entries in our data.  

In [9]:
raw_data.take(5)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.']

In the following notebooks, we will use this raw data to learn about the different Spark transformations and actions.  

## Creating an RDD using `parallelize`

Another way of creating an RDD is to parallelize an already existing list.  

In [10]:
a = range(1000)

data = sc.parallelize(a)

As we did before, we can `count()` the number of elements in the RDD.

In [11]:
data.count()

1000

As before, we can access the first few elements on our RDD.  

In [12]:
data.take(5)

[0, 1, 2, 3, 4]

## The `filter` transformation

This transformation can be applied to RDDs in order to keep just elements that satisfy a certain condition. More concretely, a function is evaluated on every element in the original RDD. The new resulting RDD will contain just those elements that make the function return `True`.

For example, imagine we want to count how many `normal.` interactions we have in our dataset. We can filter our `raw_data` RDD as follows.  

In [13]:
normal_raw_data = raw_data.filter(lambda x: 'normal.' in x)

In [14]:
normal_raw_data.count()   # 97278

97278

In [15]:
t0 = time()
normal_count = normal_raw_data.count()
tt = time() - t0
print("There are {} 'normal' interactions".format(normal_count)) 
print ("Count completed in {} seconds".format(round(tt,3)))

There are 97278 'normal' interactions
Count completed in 1.484 seconds


Notice that we have measured the elapsed time for counting the elements in the RDD. We have done this because we wanted to point out that actual (distributed) computations in Spark take place when we execute *actions* and not *transformations*. In this case `count` is the action we execute on the RDD. We can apply as many transformations as we want on a our RDD and no computation will take place until we call the first action that, in this case takes a few seconds to complete.

## The `map` transformation

By using the `map` transformation in Spark, we can apply a function to every element in our RDD. Python's lambdas are specially expressive for this particular.

In this case we want to read our data file as a CSV formatted one. We can do this by applying a lambda function to each element in the RDD as follows.

In [16]:
from pprint import pprint
csv_data = raw_data.map(lambda x: x.split(","))
t0 = time()
head_rows = csv_data.take(5)
tt = time() - t0
print ("Parse completed in {} seconds".format(round(tt,3)))
pprint(head_rows[0])

Parse completed in 0.05 seconds
['0',
 'tcp',
 'http',
 'SF',
 '181',
 '5450',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '8',
 '8',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '1.00',
 '0.00',
 '0.00',
 '9',
 '9',
 '1.00',
 '0.00',
 '0.11',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 'normal.']


In [17]:
t0 = time()
head_rows = csv_data.take(10000)
tt = time() - t0
print ("Parse completed in {} seconds".format(round(tt,3)))

Parse completed in 0.606 seconds


### Using `map` with predefined functions

Of course we can use predefined functions with `map`. Imagine we want to have each element in the RDD as a key-value pair where the key is the tag (e.g. *normal*) and the value is the whole list of elements that represents the row in the CSV formatted file. We could proceed as follows.  

In [18]:
row1 = raw_data.first()

In [21]:
type(row1), len(row1.split(',')), row1

(str,
 42,
 '0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.')

In [22]:
def parse_interaction(line):
    elems = line.split(",")
    tag = elems[-1]
    return (tag, elems)

key_csv_data = raw_data.map(parse_interaction)
head_rows = key_csv_data.take(5)
pprint(head_rows[0])

('normal.',
 ['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'])


## The `collect` action

So far we have used the actions `count` and `take`. Another basic action we need to learn is `collect`. Basically it will get all the elements in the RDD into memory for us to work with them. For this reason it has to be used with care, specially when working with large RDDs.  

In [24]:
t0 = time()
all_raw_data = raw_data.collect()
tt = time() - t0
print ("Data collected in {} seconds".format(round(tt,3)))

Data collected in 5.294 seconds


That took longer as any other action we used before, of course. Every Spark worker node that has a fragment of the RDD has to be coordinated in order to retrieve its part, and then *reduce* everything together.   

As a last example combining all the previous, we want to collect all the normal interactions as key-value pairs

In [25]:
# parse into key-value pairs
key_csv_data = raw_data.map(parse_interaction)

# filter normal key interactions
normal_key_interactions = key_csv_data.filter(lambda x: x[0] == "normal.")

# collect all
t0 = time()
all_normal = normal_key_interactions.collect()
tt = time() - t0
normal_count = len(all_normal)
print ("Data collected in {} seconds".format(round(tt,3)))
print ("There are {} 'normal' interactions".format(normal_count))

Data collected in 4.336 seconds
There are 97278 'normal' interactions


## Sampling RDDs

So far we have introduced RDD creation together with some basic transformations such as map and filter and some actions such as count, take, and collect.

This notebook will show how to sample RDDs. Regarding transformations, sample will be introduced since it will be useful in many statistical learning scenarios. Then we will compare results with the takeSample action.

In Spark, there are two sampling operations, the transformation `sample` and the action `takeSample`. By using a transformation we can tell Spark to apply successive transformation on a sample of a given RDD. By using an action we retrieve a given sample and we can have it in local memory to be used by any other standard library (e.g. Scikit-learn).  

### The `sample` transformation

The `sample` transformation takes up to three parameters. First is whether the sampling is done with replacement or not. Second is the sample size as a fraction. Finally we can optionally provide a *random seed*.  

In [28]:
raw_data_sample = raw_data.sample(False, 0.1, 1234)
sample_size = raw_data_sample.count()
total_size = raw_data.count()
print ("Sample size is {} of {}".format(sample_size, total_size))

Sample size is 49493 of 494021


But the power of sampling as a transformation comes from doing it as part of a sequence of additional transformations. This will show more powerful once we start doing aggregations and key-value pairs operations, and will be specially useful when using Spark's machine learning library MLlib.    

In the meantime, imagine we want to have an approximation of the proportion of `normal.` interactions in our dataset. We could do this by counting the total number of tags as we did in previous notebooks. However we want a quicker response and we don't need the exact answer but just an approximation. We can do it as follows.   

In [29]:
# transformations to be applied
raw_data_sample_items = raw_data_sample.map(lambda x: x.split(","))
sample_normal_tags = raw_data_sample_items.filter(lambda x: "normal." in x)

# actions + time
t0 = time()
sample_normal_tags_count = sample_normal_tags.count()
tt = time() - t0

sample_normal_ratio = sample_normal_tags_count / float(sample_size)
print ("The ratio of 'normal' interactions is {}".format(round(sample_normal_ratio,3)) )
print ("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.195
Count done in 1.675 seconds


Let's compare this with calculating the ratio without sampling.

In [30]:
# transformations to be applied
raw_data_items = raw_data.map(lambda x: x.split(","))
normal_tags = raw_data_items.filter(lambda x: "normal." in x)

# actions + time
t0 = time()
normal_tags_count = normal_tags.count()
tt = time() - t0

normal_ratio = normal_tags_count / float(total_size)
print ("The ratio of 'normal' interactions is {}".format(round(normal_ratio,3)) )
print ("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.197
Count done in 3.11 seconds


We can see a gain in time. The more transformations we apply after the sampling the bigger this gain. This is because without sampling all the transformations are applied to the complete set of data.  

### The `takeSample` action  

If what we need is to grab a sample of raw data from our RDD into local memory in order to be used by other non-Spark libraries, `takeSample` can be used.  

The syntax is very similar, but in this case we specify the number of items instead of the sample size as a fraction of the complete data size.  

In [32]:
rows_to_sample = 100000

t0 = time()
raw_data_sample = raw_data.takeSample(False, rows_to_sample, 1234)
normal_data_sample = [x.split(",") for x in raw_data_sample if "normal." in x]
tt = time() - t0

normal_sample_size = len(normal_data_sample)

normal_ratio = normal_sample_size / rows_to_sample
print ("The ratio of 'normal' interactions is {}".format(normal_ratio))
print ("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.19633
Count done in 3.049 seconds


The process was very similar as before. We obtained a sample of about 10 percent of the data, and then filter and split.  

However, it took longer, even with a slightly smaller sample. The reason is that Spark just distributed the execution of the sampling process. The filtering and splitting of the results were done locally in a single node. 

##  Set operations on RDDs

Spark supports many of the operations we have in mathematical sets, such as union and intersection, even when the RDDs themselves are not properly sets. It is important to note that these operations require that the RDDs being operated on are of the same type.  

Set operations are quite straightforward to understand as it work as expected. The only consideration comes from the fact that RDDs are not real sets, and therefore operations such as the union of RDDs doesn't remove duplicates. In this notebook we will have a brief look at `subtract`, `distinct`, and `cartesian`.       

### Getting attack interactions using `subtract`

For illustrative purposes, imagine we already have our RDD with non attack (normal) interactions from some previous analysis.

We can obtain attack interactions by subtracting normal ones from the original unfiltered RDD as follows.  

In [33]:
normal_raw_data = raw_data.filter(lambda x: "normal." in x)
attack_raw_data = raw_data.subtract(normal_raw_data)

In [34]:
normal_raw_data.count(), attack_raw_data.count()

(97278, 396743)

In [35]:
attack_raw_data.take(5)

['305,tcp,telnet,SF,1735,2766,0,0,0,3,0,1,2,1,0,0,1,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,2,4,1.00,0.00,0.50,0.50,0.00,0.00,0.00,0.00,buffer_overflow.',
 '0,tcp,telnet,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,5,0.83,1.00,0.00,0.00,0.83,0.33,0.00,5,6,1.00,0.00,0.20,0.33,1.00,0.83,0.00,0.00,neptune.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,123,0.48,0.01,0.48,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,133,0.52,0.01,0.52,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,183,0.72,0.01,0.72,0.00,0.00,0.00,0.00,0.00,smurf.']

### Protocol and service combinations using `cartesian`

We can compute the Cartesian product between two RDDs by using the cartesian transformation. It returns all possible pairs of elements between two RDDs. In our case we will use it to generate all the possible combinations between service and protocol in our network interactions.

First of all we need to isolate each collection of values in two separate RDDs. For that we will use distinct on the CSV-parsed dataset. From the [dataset description](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names) we know that protocol is the second column and service is the third (tag is the last one and not the first as appears in the page).

In [39]:
# get attach type
csv_data = raw_data.map(lambda x: x.split(","))
attack_types = csv_data.map(lambda x: x[-1]).distinct()
attack_types.collect()

['normal.',
 'buffer_overflow.',
 'loadmodule.',
 'perl.',
 'neptune.',
 'smurf.',
 'guess_passwd.',
 'pod.',
 'teardrop.',
 'portsweep.',
 'ipsweep.',
 'land.',
 'ftp_write.',
 'back.',
 'imap.',
 'satan.',
 'phf.',
 'nmap.',
 'multihop.',
 'warezmaster.',
 'warezclient.',
 'spy.',
 'rootkit.']

In [40]:
# get protocol
csv_data = raw_data.map(lambda x: x.split(","))
protocols = csv_data.map(lambda x: x[1]).distinct()
protocols.collect()

['tcp', 'udp', 'icmp']

In [37]:
services = csv_data.map(lambda x: x[2]).distinct()
services.collect()

['http',
 'smtp',
 'finger',
 'domain_u',
 'auth',
 'telnet',
 'ftp',
 'eco_i',
 'ntp_u',
 'ecr_i',
 'other',
 'private',
 'pop_3',
 'ftp_data',
 'rje',
 'time',
 'mtp',
 'link',
 'remote_job',
 'gopher',
 'ssh',
 'name',
 'whois',
 'domain',
 'login',
 'imap4',
 'daytime',
 'ctf',
 'nntp',
 'shell',
 'IRC',
 'nnsp',
 'http_443',
 'exec',
 'printer',
 'efs',
 'courier',
 'uucp',
 'klogin',
 'kshell',
 'echo',
 'discard',
 'systat',
 'supdup',
 'iso_tsap',
 'hostnames',
 'csnet_ns',
 'pop_2',
 'sunrpc',
 'uucp_path',
 'netbios_ns',
 'netbios_ssn',
 'netbios_dgm',
 'sql_net',
 'vmnet',
 'bgp',
 'Z39_50',
 'ldap',
 'netstat',
 'urh_i',
 'X11',
 'urp_i',
 'pm_dump',
 'tftp_u',
 'tim_i',
 'red_i']

In [41]:
# do the cartesian product.
product = protocols.cartesian(services).collect()
print ("There are {} combinations of protocol X service".format(len(product)))

There are 198 combinations of protocol X service


Obviously, for such small RDDs doesn't really make sense to use Spark cartesian product. We could have perfectly collected the values after using `distinct` and do the cartesian product locally. Moreover, `distinct` and `cartesian` are expensive operations so they must be used with care when the operating datasets are large.   

##  Data aggregations on RDDs

We can aggregate RDD data in Spark by using three different actions: reduce, fold, and aggregate. The last one is the more general one and someway includes the first two.


### Inspecting interaction duration by tag

Both `fold` and `reduce` take a function as an argument that is applied to two elements of the RDD. The `fold` action differs from `reduce` in that it gets and additional initial *zero value* to be used for the initial call. This value should be the identity element for the function provided.  

As an example, imagine we want to know the total duration of our interactions for normal and attack interactions. We can use `reduce` as follows.   

In [42]:
# parse data
csv_data = raw_data.map(lambda x: x.split(","))

# separate into different RDDs
normal_csv_data = csv_data.filter(lambda x: x[41]=="normal.")
attack_csv_data = csv_data.filter(lambda x: x[41]!="normal.")

The function that we pass to `reduce` gets and returns elements of the same type of the RDD. If we want to sum durations we need to extract that element into a new RDD.  

In [43]:
normal_duration_data = normal_csv_data.map(lambda x: int(x[0]))
attack_duration_data = attack_csv_data.map(lambda x: int(x[0]))

In [44]:
total_normal_duration = normal_duration_data.reduce(lambda x, y: x + y)
total_attack_duration = attack_duration_data.reduce(lambda x, y: x + y)

In [45]:
print ("Total duration for 'normal' interactions is {}".\
    format(total_normal_duration))
print ("Total duration for 'attack' interactions is {}".\
    format(total_attack_duration))

Total duration for 'normal' interactions is 21075991
Total duration for 'attack' interactions is 2626792


We can go further and use counts to calculate duration means.  

In [46]:
normal_count = normal_duration_data.count()
attack_count = attack_duration_data.count()

print ("Mean duration for 'normal' interactions is {}".\
    format(round(total_normal_duration/float(normal_count),3)))
print ("Mean duration for 'attack' interactions is {}".\
    format(round(total_attack_duration/float(attack_count),3)))

Mean duration for 'normal' interactions is 216.657
Mean duration for 'attack' interactions is 6.621


### A better way, using `aggregate`  

The `aggregate` action frees us from the constraint of having the return be the same type as the RDD we are working on. Like with `fold`, we supply an initial zero value of the type we want to return. Then we provide two functions. The first one is used to combine the elements from our RDD with the accumulator. The second function is needed to merge two accumulators. Let's see it in action calculating the mean we did before.  

In [47]:
normal_sum_count = normal_duration_data.aggregate(
    (0,0), # the initial value
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), # combine value with acc
    (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])) # combine accumulators
)

print ("Mean duration for 'normal' interactions is {}".\
    format(round(normal_sum_count[0]/float(normal_sum_count[1]),3)))

Mean duration for 'normal' interactions is 216.657


## Working with key/value pair RDDs

Spark provides specific functions to deal with RDDs which elements are key/value pairs. They are usually used to perform aggregations and other processings by key.  

In this notebook we will show how, by working with key/value pairs, we can process our network interactions dataset in a more practical and powerful way than that used in previous notebooks. Key/value pair aggregations will show to be particularly effective when trying to explore each type of tag in our network attacks, in an individual way.  

### Creating a pair RDD for interaction types

In this notebook we want to do some exploratory data analysis on our network interactions dataset. More concretely we want to profile each network interaction type in terms of some of its variables such as duration. In order to do so, we first need to create the RDD suitable for that, where each interaction is parsed as a CSV row representing the value, and is put together with its corresponding tag as a key.  

Normally we create key/value pair RDDs by applying a function using `map` to the original data. This function returns the corresponding pair for a given RDD element. We can proceed as follows.  

In [48]:
csv_data = raw_data.map(lambda x: x.split(","))
key_value_data = csv_data.map(lambda x: (x[41], x)) # x[41] contains the network interaction tag

In [49]:
key_value_data.first()

('normal.',
 ['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'])

### Data aggregations with key/value pair RDDs

We can use all the transformations and actions available for normal RDDs with key/value pair RDDs. We just need to make the functions work with pair elements. Additionally, Spark provides specific functions to work with RDDs containing pair elements. They are very similar to those available for general RDDs.  

For example, we have a `reduceByKey` transformation that we can use as follows to calculate the total duration of each network interaction type.  

In [50]:
key_value_duration = csv_data.map(lambda x: (x[41], float(x[0]))) 
durations_by_key = key_value_duration.reduceByKey(lambda x, y: x + y)

durations_by_key.collect()

[('normal.', 21075991.0),
 ('buffer_overflow.', 2751.0),
 ('loadmodule.', 326.0),
 ('perl.', 124.0),
 ('neptune.', 0.0),
 ('smurf.', 0.0),
 ('guess_passwd.', 144.0),
 ('pod.', 0.0),
 ('teardrop.', 0.0),
 ('portsweep.', 1991911.0),
 ('ipsweep.', 43.0),
 ('land.', 0.0),
 ('ftp_write.', 259.0),
 ('back.', 284.0),
 ('imap.', 72.0),
 ('satan.', 64.0),
 ('phf.', 18.0),
 ('nmap.', 0.0),
 ('multihop.', 1288.0),
 ('warezmaster.', 301.0),
 ('warezclient.', 627563.0),
 ('spy.', 636.0),
 ('rootkit.', 1008.0)]

We have a specific counting action for key/value pairs.  

In [51]:
counts_by_key = key_value_data.countByKey()
counts_by_key

defaultdict(int,
            {'normal.': 97278,
             'buffer_overflow.': 30,
             'loadmodule.': 9,
             'perl.': 3,
             'neptune.': 107201,
             'smurf.': 280790,
             'guess_passwd.': 53,
             'pod.': 264,
             'teardrop.': 979,
             'portsweep.': 1040,
             'ipsweep.': 1247,
             'land.': 21,
             'ftp_write.': 8,
             'back.': 2203,
             'imap.': 12,
             'satan.': 1589,
             'phf.': 4,
             'nmap.': 231,
             'multihop.': 7,
             'warezmaster.': 20,
             'warezclient.': 1020,
             'spy.': 2,
             'rootkit.': 10})

### Using `combineByKey`

This is the most general of the per-key aggregation functions. Most of the other per-key combiners are implemented using it. We can think about it as the `aggregate` equivalent since it allows the user to return values that are not the same type as our input data.

For example, we can use it to calculate per-type average durations as follows.  

In [ ]:
sum_counts = key_value_duration.combineByKey(
    (lambda x: (x, 1)), # the initial value, with value x and count 1
    (lambda acc, value: (acc[0]+value, acc[1]+1)), # how to combine a pair value with the accumulator: sum value, and increment count
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) # combine accumulators
)

In [82]:
type(sum_counts)

pyspark.rdd.PipelinedRDD

In [81]:
sum_counts.collect()

[('normal.', (21075991.0, 97278)),
 ('buffer_overflow.', (2751.0, 30)),
 ('loadmodule.', (326.0, 9)),
 ('perl.', (124.0, 3)),
 ('neptune.', (0.0, 107201)),
 ('smurf.', (0.0, 280790)),
 ('guess_passwd.', (144.0, 53)),
 ('pod.', (0.0, 264)),
 ('teardrop.', (0.0, 979)),
 ('portsweep.', (1991911.0, 1040)),
 ('ipsweep.', (43.0, 1247)),
 ('land.', (0.0, 21)),
 ('ftp_write.', (259.0, 8)),
 ('back.', (284.0, 2203)),
 ('imap.', (72.0, 12)),
 ('satan.', (64.0, 1589)),
 ('phf.', (18.0, 4)),
 ('nmap.', (0.0, 231)),
 ('multihop.', (1288.0, 7)),
 ('warezmaster.', (301.0, 20)),
 ('warezclient.', (627563.0, 1020)),
 ('spy.', (636.0, 2)),
 ('rootkit.', (1008.0, 10))]

We can see that the arguments are pretty similar to those passed to `aggregate` in the previous notebook. The result associated to each type is in the form of a pair. If we want to actually get the averages, we need to do the division before collecting the results.  

In [83]:
sum_counts.take(2)

[('normal.', (21075991.0, 97278)), ('buffer_overflow.', (2751.0, 30))]

In [84]:
type(sum_counts)

pyspark.rdd.PipelinedRDD

In [104]:
duration_means_by_type = sum_counts.map(lambda x: (x[0], round(x[1][0]/x[1][1],3)) ).collect()

In [105]:
type(duration_means_by_type)

list

In [106]:
duration_means_by_type[:2]

[('normal.', 216.657), ('buffer_overflow.', 91.7)]

In [110]:
duration_means_by_type.sort(key=lambda tup: tup[1], reverse=True)  # sorts in place

In [111]:
# Print them sorted
for tag,avg in duration_means_by_type:
    print (tag, avg)

portsweep. 1915.299
warezclient. 615.258
spy. 318.0
normal. 216.657
multihop. 184.0
rootkit. 100.8
buffer_overflow. 91.7
perl. 41.333
loadmodule. 36.222
ftp_write. 32.375
warezmaster. 15.05
imap. 6.0
phf. 4.5
guess_passwd. 2.717
back. 0.129
satan. 0.04
ipsweep. 0.034
neptune. 0.0
smurf. 0.0
pod. 0.0
teardrop. 0.0
land. 0.0
nmap. 0.0


## Spark SQL and Data Frames

This notebook will introduce Spark capabilities to deal with data in a structured way. Basically, everything turns around the concept of *Data Frame* and using *SQL language* to query them. We will see how the data frame abstraction, very popular in other data analytics ecosystems (e.g. R and Python/Pandas), it is very powerful when performing exploratory data analysis. In fact, it is very easy to express data queries when used together with the SQL language. Moreover, Spark distributes this column-based data structure transparently, in order to make the querying process as efficient as possible.  

### Getting a Data Frame

A Spark `DataFrame` is a distributed collection of data organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R or Pandas. They can be constructed from a wide array of sources such as a existing RDD in our case.

The entry point into all SQL functionality in Spark is the `SQLContext` class. To create a basic instance, all we need is a `SparkContext` reference. Since we are running Spark in shell mode (using pySpark) we can use the global context object `sc` for this purpose.  

In [10]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

### Inferring the schema

With a `SQLContext`, we are ready to create a `DataFrame` from our existing RDD. But first we need to tell Spark SQL the schema in our data.   

Spark SQL can convert an RDD of `Row` objects to a `DataFrame`. Rows are constructed by passing a list of key/value pairs as *kwargs* to the `Row` class. The keys define the column names, and the types are inferred by looking at the first row. Therefore, it is important that there is no missing data in the first row of the RDD in order to properly infer the schema.

In our case, we first need to split the comma separated data, and then use the information in KDD's 1999 task description to obtain the [column names](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names).  

Once we have our RDD of Row we can infer and register the schema.

In [11]:
from pyspark.sql import Row

csv_data = raw_data.map(lambda l: l.split(","))
row_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5])
    )
)

In [12]:
type(raw_data), type(csv_data), type(row_data)

(pyspark.rdd.RDD, pyspark.rdd.PipelinedRDD, pyspark.rdd.PipelinedRDD)

In [13]:
raw_data.take(2)  # list of strings

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.']

In [14]:
csv_data.take(2)  # list of list of strings

[['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '239',
  '486',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '19',
  '19',
  '1.00',
  '0.00',
  '0.05',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.']]

In [15]:
row_data.take(2)  # list of Rows

[Row(dst_bytes=5450, duration=0, flag='SF', protocol_type='tcp', service='http', src_bytes=181),
 Row(dst_bytes=486, duration=0, flag='SF', protocol_type='tcp', service='http', src_bytes=239)]

In [16]:
type(row_data.take(2))

list

In [17]:
interactions_df = sqlContext.createDataFrame(row_data)
# interactions_df = row_data.toDF()
interactions_df.registerTempTable("interactions")

In [18]:
type(interactions_df)

pyspark.sql.dataframe.DataFrame

In [19]:
interactions_df.head(5)

[Row(dst_bytes=5450, duration=0, flag='SF', protocol_type='tcp', service='http', src_bytes=181),
 Row(dst_bytes=486, duration=0, flag='SF', protocol_type='tcp', service='http', src_bytes=239),
 Row(dst_bytes=1337, duration=0, flag='SF', protocol_type='tcp', service='http', src_bytes=235),
 Row(dst_bytes=1337, duration=0, flag='SF', protocol_type='tcp', service='http', src_bytes=219),
 Row(dst_bytes=2032, duration=0, flag='SF', protocol_type='tcp', service='http', src_bytes=217)]

In [20]:
idf = row_data.toDF()

In [21]:
type(idf)

pyspark.sql.dataframe.DataFrame

In [22]:
idf.head(2)

[Row(dst_bytes=5450, duration=0, flag='SF', protocol_type='tcp', service='http', src_bytes=181),
 Row(dst_bytes=486, duration=0, flag='SF', protocol_type='tcp', service='http', src_bytes=239)]

In [23]:
interactions_df.printSchema()

root
 |-- dst_bytes: long (nullable = true)
 |-- duration: long (nullable = true)
 |-- flag: string (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- src_bytes: long (nullable = true)



In [24]:
ir = idf.take(2)[0]

In [25]:
ir.dst_bytes, ir.src_bytes

(5450, 181)

### Queries as `DataFrame` operations

Spark `DataFrame` provides a domain-specific language for structured data manipulation. This language includes methods we can concatenate in order to do selection, filtering, grouping, etc. For example, let's say we want to count how many interactions are there for each protocol type. We can proceed as follows.  

#### tips
* [order-by (sort)](https://stackoverflow.com/questions/34514545/spark-dataframe-groupby-and-sort-in-the-descending-order-pyspark)

* [rename column](https://stackoverflow.com/questions/34077353/how-to-change-dataframe-column-names-in-pyspark)

Now we can run SQL queries over our data frame that has been registered as a table.

In [26]:
# %%time
# Select tcp network interactions with more than 1 second duration and no transfer from destination
tcp_interactions = sqlContext.sql("""
    SELECT duration, dst_bytes FROM interactions WHERE protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0
""")
tcp_interactions.show(5)
# type(tcp_interactions)   # pyspark.sql.dataframe.DataFrame

+--------+---------+
|duration|dst_bytes|
+--------+---------+
|    5057|        0|
|    5059|        0|
|    5051|        0|
|    5056|        0|
|    5051|        0|
+--------+---------+
only showing top 5 rows



In [27]:
type(tcp_interactions)

pyspark.sql.dataframe.DataFrame

In [28]:
tcp_interactions.count()

139

In [29]:
for row in tcp_interactions.collect()[:5]:
    print (f"Duration: {row.duration}, Dest. bytes: {row.dst_bytes}")

Duration: 5057, Dest. bytes: 0
Duration: 5059, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0
Duration: 5056, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0


In [49]:
%%time
# where clause
# limit 10
interactions_df\
    .select("protocol_type", "duration", "dst_bytes")\
    .filter(" dst_bytes > 1000")\
    .filter(" duration > 1000")\
    .show(10)

+-------------+--------+---------+
|protocol_type|duration|dst_bytes|
+-------------+--------+---------+
|          tcp|   12454|    21181|
|          tcp|   13368|    58843|
|          tcp|   10350|     5780|
|          tcp|   10409|     2925|
|          tcp|   14918|    25239|
|          tcp|   10039|    12602|
|          tcp|   15127|   185137|
|          tcp|   25602|     8518|
|          tcp|   13120|   195153|
|          tcp|    2399|   460350|
+-------------+--------+---------+
only showing top 10 rows

CPU times: user 5.03 ms, sys: 174 µs, total: 5.2 ms
Wall time: 1.6 s


In [57]:
%%time
# group by
interactions_df\
    .select("protocol_type")\
    .groupBy("protocol_type").count()\
    .show()

+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|190065|
|          udp| 20354|
|         icmp|283602|
+-------------+------+

CPU times: user 10.5 ms, sys: 14.2 ms, total: 24.7 ms
Wall time: 10 s


In [59]:
%%time
# column alias
# https://stackoverflow.com/questions/33516490/column-alias-after-groupby-in-pyspark
interactions_df\
    .select("protocol_type")\
    .groupBy("protocol_type").count()\
    .select("protocol_type", col("count").alias("Prot_Cnt"))\
    .show()

+-------------+--------+
|protocol_type|Prot_Cnt|
+-------------+--------+
|          tcp|  190065|
|          udp|   20354|
|         icmp|  283602|
+-------------+--------+

CPU times: user 16 ms, sys: 9.7 ms, total: 25.8 ms
Wall time: 10.8 s


We can use this to perform some [exploratory data analysis](http://en.wikipedia.org/wiki/Exploratory_data_analysis). Let's count how many attack and normal interactions we have. First we need to add the label column to our data.    

In [36]:
# add column (transform)
def get_label_type(label):
    if label!="normal.":
        return "attack"
    else:
        return "normal"
    
row_labeled_data = csv_data.map(
    lambda p: Row(
        duration=int(p[0]), 
        protocol_type=p[1],
        service=p[2],
        flag=p[3],
        src_bytes=int(p[4]),
        dst_bytes=int(p[5]),
        label=get_label_type(p[41])
    )
)
interactions_labeled_df = sqlContext.createDataFrame(row_labeled_data)

This time we don't need to register the schema since we are going to use the OO query interface.

Let's check the previous actually works by counting attack and normal data in our data frame.

In [37]:
%%time
# group by
interactions_labeled_df\
    .select("label")\
    .groupBy("label").count()\
    .show()

+------+------+
| label| count|
+------+------+
|normal| 97278|
|attack|396743|
+------+------+

CPU times: user 21.8 ms, sys: 276 µs, total: 22.1 ms
Wall time: 9.85 s


Now we want to count them by label and protocol type, in order to see how important the protocol type is to detect when an interaction is or not an attack.  

In [61]:
%%time
# order by
interactions_labeled_df\
    .select("label", "protocol_type")\
    .groupBy("label", "protocol_type").count()\
    .sort(col("label"), desc("protocol_type"))\
    .show()

+------+-------------+------+
| label|protocol_type| count|
+------+-------------+------+
|attack|          udp|  1177|
|attack|          tcp|113252|
|attack|         icmp|282314|
|normal|          udp| 19177|
|normal|          tcp| 76813|
|normal|         icmp|  1288|
+------+-------------+------+

CPU times: user 19.5 ms, sys: 8 ms, total: 27.5 ms
Wall time: 10.4 s


At first sight it seems that *udp* interactions are in lower proportion between network attacks versus other protocol types.  

And we can do much more sophisticated groupings. For example, add to the previous a "split" based on data transfer from target. 

In [65]:
%%time
interactions_labeled_df\
    .select("label", "protocol_type", "dst_bytes")\
    .groupBy("label", "protocol_type", interactions_labeled_df.dst_bytes==0).count()\
    .select("label", "protocol_type", col("(dst_bytes = 0)").alias("dst_0byte"), "count")\
    .sort(col("label"), desc("protocol_type"), desc("dst_0byte"))\
    .show()

+------+-------------+---------+------+
| label|protocol_type|dst_0byte| count|
+------+-------------+---------+------+
|attack|          udp|     true|  1166|
|attack|          udp|    false|    11|
|attack|          tcp|     true|110583|
|attack|          tcp|    false|  2669|
|attack|         icmp|     true|282314|
|normal|          udp|     true|  3594|
|normal|          udp|    false| 15583|
|normal|          tcp|     true|  9313|
|normal|          tcp|    false| 67500|
|normal|         icmp|     true|  1288|
+------+-------------+---------+------+

CPU times: user 24 ms, sys: 9 ms, total: 33 ms
Wall time: 10.6 s


We see how relevant is this new split to determine if a network interaction is an attack.  

We will stop here, but we can see how powerful this type of queries are in order to explore our data. Actually we can replicate all the splits we saw in previous notebooks, when introducing classification trees, just by selecting, groping, and filtering our dataframe. For a more detailed (but less real-world) list of Spark's `DataFrame` operations and data sources, have a look at the official documentation [here](https://spark.apache.org/docs/latest/sql-programming-guide.html#dataframe-operations).    

## MLlib: Basic Statistics and Exploratory Data Analysis


So far we have used different map and aggregation functions, on simple and key/value pair RDD's, in order to get simple statistics that help us understand our datasets. In this notebook we will introduce Spark's machine learning library [MLlib](https://spark.apache.org/docs/latest/mllib-guide.html) through its basic statistics functionality in order to better understand our dataset. We will use the reduced 10-percent [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) datasets through the notebook.  

### Local vectors

A [local vector](https://spark.apache.org/docs/latest/mllib-data-types.html#local-vector) is often used as a base type for RDDs in Spark MLlib. A local vector has integer-typed and 0-based indices and double-typed values, stored on a single machine. MLlib supports two types of local vectors: dense and sparse. A dense vector is backed by a double array representing its entry values, while a sparse vector is backed by two parallel arrays: indices and values. 

For dense vectors, MLlib uses either Python *lists* or the *NumPy* `array` type. The later is recommended, so you can simply pass NumPy arrays around.  

For sparse vectors, users can construct a `SparseVector` object from MLlib or pass *SciPy* `scipy.sparse` column vectors if SciPy is available in their environment. The easiest way to create sparse vectors is to use the factory methods implemented in `Vectors`.  

#####  An RDD of dense vectors

Let's represent each network interaction in our dataset as a dense vector. For that we will use the *NumPy* `array` type.  

In [9]:
raw_data.first()

'0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.'

In [10]:
import numpy as np

def parse_interaction(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [1,2,3,41]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return np.array([float(x) for x in clean_line_split])

vector_data = raw_data.map(parse_interaction)

In [11]:
type(vector_data)

pyspark.rdd.PipelinedRDD

In [12]:
vector_data.take(2)

[array([0.00e+00, 1.81e+02, 5.45e+03, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 8.00e+00, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 9.00e+00, 9.00e+00,
        1.00e+00, 0.00e+00, 1.10e-01, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00]),
 array([0.00e+00, 2.39e+02, 4.86e+02, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 8.00e+00, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 1.90e+01, 1.90e+01,
        1.00e+00, 0.00e+00, 5.00e-02, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00])]

### Summary statistics

Spark's MLlib provides column summary statistics for `RDD[Vector]` through the function [`colStats`](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.stat.Statistics.colStats) available in [`Statistics`](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.stat.Statistics). The method returns an instance of [`MultivariateStatisticalSummary`](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.stat.MultivariateStatisticalSummary), which contains the column-wise *max*, *min*, *mean*, *variance*, and *number of nonzeros*, as well as the *total count*.  

In [13]:
from pyspark.mllib.stat import Statistics 
from math import sqrt 

# Compute column summary statistics.
summary = Statistics.colStats(vector_data)

count = summary.count()
# look at 1st column  
mean = round(summary.mean()[0],3)
std = round(sqrt(summary.variance()[0]),3)
max_ = round(summary.max()[0],3)
min_ = round(summary.min()[0],3)
ct_non_zero = summary.numNonzeros()[0]


In [14]:
print(f"""
Duration Statistics:
    Mean: {mean}
    St. deviation: {std}
    Max value: {max_}
    Min value: {min_}
    Total value count: {count}
    Number of non-zero values: {ct_non_zero}
""")


Duration Statistics:
    Mean: 47.979
    St. deviation: 707.746
    Max value: 58329.0
    Min value: 0.0
    Total value count: 494021
    Number of non-zero values: 12350.0



In [16]:
len(summary.mean()), summary.mean()

(38, array([4.79793025e+01, 3.02561030e+03, 8.68532425e+02, 4.45325199e-05,
        6.43292492e-03, 1.41694381e-05, 3.45187755e-02, 1.51815409e-04,
        1.48246734e-01, 1.02121165e-02, 1.11331300e-04, 3.64356981e-05,
        1.13517442e-02, 1.08294992e-03, 1.09307094e-04, 1.00805431e-03,
        0.00000000e+00, 0.00000000e+00, 1.38658073e-03, 3.32285690e+02,
        2.92906557e+02, 1.76686659e-01, 1.76608808e-01, 5.74334087e-02,
        5.77189431e-02, 7.91547343e-01, 2.09823874e-02, 2.89968038e-02,
        2.32470778e+02, 1.88665670e+02, 7.53779698e-01, 3.09057307e-02,
        6.01934756e-01, 6.68350131e-03, 1.76753964e-01, 1.76442621e-01,
        5.81176104e-02, 5.74116687e-02]))

#### Summary statistics by label  

The interesting part of summary statistics, in our case, comes from being able to obtain them by the type of network attack or 'label' in our dataset. By doing so we will be able to better characterise our dataset dependent variable in terms of the independent variables range of values.  

If we want to do such a thing we could filter our RDD containing labels as keys and vectors as values. For that we just need to adapt our `parse_interaction` function to return a tuple with both elements.    

In [17]:
def parse_interaction_with_key(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [1,2,3,41]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return (line_split[41], np.array([float(x) for x in clean_line_split]))

label_vector_data = raw_data.map(parse_interaction_with_key)

In [18]:
label_vector_data.take(2)

[('normal.', array([0.00e+00, 1.81e+02, 5.45e+03, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 8.00e+00, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 9.00e+00, 9.00e+00,
         1.00e+00, 0.00e+00, 1.10e-01, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00])),
 ('normal.', array([0.00e+00, 2.39e+02, 4.86e+02, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 8.00e+00, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 1.90e+01, 1.90e+01,
         1.00e+00, 0.00e+00, 5.00e-02, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00]))]

In [19]:
normal_label_data = label_vector_data.filter(lambda x: x[0]=="normal.")
normal_summary = Statistics.colStats(normal_label_data.values())

mean = round(normal_summary.mean()[0],3)
std = round(sqrt(normal_summary.variance()[0]),3)
max_ = round(normal_summary.max()[0],3)
min_ = round(normal_summary.min()[0],3)
count = normal_summary.count()
ct_non_zero = normal_summary.numNonzeros()[0]

In [20]:
print(f"""
Duration Statistics:
    Mean: {mean}
    St. deviation: {std}
    Max value: {max_}
    Min value: {min_}
    Total value count: {count}
    Number of non-zero values: {ct_non_zero}
""")


Duration Statistics:
    Mean: 216.657
    St. deviation: 1359.213
    Max value: 58329.0
    Min value: 0.0
    Total value count: 97278
    Number of non-zero values: 11690.0



Instead of working with a key/value pair we could have just filter our raw data split using the label in column 41. Then we can parse the results as we did before. This will work as well. However having our data organised as key/value pairs will open the door to better manipulations. Since `values()` is a transformation on an RDD, and not an action, we don't perform any computation until we call `colStats` anyway.  

But lets wrap this within a function so we can reuse it with any label.

In [21]:
def summary_by_label(raw_data, label):
    label_vector_data = raw_data.map(parse_interaction_with_key).filter(lambda x: x[0]==label)
    return Statistics.colStats(label_vector_data.values())

In [22]:
normal_sum = summary_by_label(raw_data, "normal.")

In [23]:
mean = round(normal_sum.mean()[0],3)
std = round(sqrt(normal_sum.variance()[0]),3)
max_ = round(normal_sum.max()[0],3)
min_ = round(normal_sum.min()[0],3)
count = normal_sum.count()
ct_non_zero = normal_sum.numNonzeros()[0]

In [24]:
print(f"""
Duration Statistics for label {label}:
    Mean: {mean}
    St. deviation: {std}
    Max value: {max_}
    Min value: {min_}
    Total value count: {count}
    Number of non-zero values: {ct_non_zero}
""")


Duration Statistics:
    Mean: 216.657
    St. deviation: 1359.213
    Max value: 58329.0
    Min value: 0.0
    Total value count: 97278
    Number of non-zero values: 11690.0



Let's try now with some network attack. We have all of them listed [here](http://kdd.ics.uci.edu/databases/kddcup99/training_attack_types).  

In [27]:
label = "guess_passwd."
label_summary = summary_by_label(raw_data, label)

mean = round(label_summary.mean()[0],3)
std = round(sqrt(label_summary.variance()[0]),3)
max_ = round(label_summary.max()[0],3)
min_ = round(label_summary.min()[0],3)
count = label_summary.count()
ct_non_zero = label_summary.numNonzeros()[0]

print(f"""
Duration Statistics for label {label}:
    Mean: {mean}
    St. deviation: {std}
    Max value: {max_}
    Min value: {min_}
    Total value count: {count}
    Number of non-zero values: {ct_non_zero}
""")


Duration Statistics for label guess_passwd.:
    Mean: 2.717
    St. deviation: 11.88
    Max value: 60.0
    Min value: 0.0
    Total value count: 53
    Number of non-zero values: 4.0



We can see that this type of attack is shorter in duration than a normal interaction. We could build a table with duration statistics for each type of interaction in our dataset. First we need to get a list of labels as described in the first line [here](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names).     

In [28]:
label_list = ["back.","buffer_overflow.","ftp_write.","guess_passwd.",
              "imap.","ipsweep.","land.","loadmodule.","multihop.",
              "neptune.","nmap.","normal.","perl.","phf.","pod.","portsweep.",
              "rootkit.","satan.","smurf.","spy.","teardrop.","warezclient.",
              "warezmaster."]

In [29]:
stats_by_label = [(label, summary_by_label(raw_data, label)) for label in label_list]

Now we get the duration column, first in our dataset (i.e. index 0).

In [30]:
duration_by_label = [ 
    (stat[0], np.array([float(stat[1].mean()[0]), float(sqrt(stat[1].variance()[0])), float(stat[1].min()[0]), float(stat[1].max()[0]), int(stat[1].count())])) 
    for stat in stats_by_label]

That we can put into a Pandas data frame.

In [31]:
import pandas as pd
pd.set_option('display.max_columns', 50)

stats_by_label_df = pd.DataFrame.from_items(duration_by_label, columns=["Mean", "Std Dev", "Min", "Max", "Count"], orient='index')

/home/gong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  after removing the cwd from sys.path.


In [32]:
print ("Duration statistics, by label")
stats_by_label_df

Duration statistics, by label


,Mean,Std Dev,Min,Max,Count
back.,0.128915,1.110062,0.0,14.0,2203.0
buffer_overflow.,91.700000,97.514685,0.0,321.0,30.0
ftp_write.,32.375000,47.449033,0.0,134.0,8.0
guess_passwd.,2.716981,11.879811,0.0,60.0,53.0
imap.,6.000000,14.174240,0.0,41.0,12.0
ipsweep.,0.034483,0.438439,0.0,7.0,1247.0
land.,0.000000,0.000000,0.0,0.0,21.0
loadmodule.,36.222222,41.408869,0.0,103.0,9.0
multihop.,184.000000,253.851006,0.0,718.0,7.0
neptune.,0.000000,0.000000,0.0,0.0,107201.0


In order to reuse this code and get a dataframe from any variable in our dataset we will define a function.  

In [33]:
def get_variable_stats_df(stats_by_label, column_i):
    column_stats_by_label = [
        (stat[0], np.array([float(stat[1].mean()[column_i]), float(sqrt(stat[1].variance()[column_i])), float(stat[1].min()[column_i]), float(stat[1].max()[column_i]), int(stat[1].count())])) 
        for stat in stats_by_label
    ]
    return pd.DataFrame.from_items(column_stats_by_label, columns=["Mean", "Std Dev", "Min", "Max", "Count"], orient='index')

In [34]:
# Let's try for duration again.
column_i = 0   # for duration
get_variable_stats_df(stats_by_label,column_i)

/home/gong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  


,Mean,Std Dev,Min,Max,Count
back.,0.128915,1.110062,0.0,14.0,2203.0
buffer_overflow.,91.700000,97.514685,0.0,321.0,30.0
ftp_write.,32.375000,47.449033,0.0,134.0,8.0
guess_passwd.,2.716981,11.879811,0.0,60.0,53.0
imap.,6.000000,14.174240,0.0,41.0,12.0
ipsweep.,0.034483,0.438439,0.0,7.0,1247.0
land.,0.000000,0.000000,0.0,0.0,21.0
loadmodule.,36.222222,41.408869,0.0,103.0,9.0
multihop.,184.000000,253.851006,0.0,718.0,7.0
neptune.,0.000000,0.000000,0.0,0.0,107201.0


In [40]:
# Now for the next numeric column in the dataset, src_bytes.

col_i_dict = {"src_bytes":1}
col_name = "src_bytes"
col_i = col_i_dict[col_name]
print(f"{col_name} statistics by label")
get_variable_stats_df(stats_by_label,col_i)

src_bytes statistics by label


/home/gong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  


,Mean,Std Dev,Min,Max,Count
back.,54156.355878,3.159360e+03,13140.0,54540.0,2203.0
buffer_overflow.,1400.433333,1.337133e+03,0.0,6274.0,30.0
ftp_write.,220.750000,2.677476e+02,0.0,676.0,8.0
guess_passwd.,125.339623,3.037860e+00,104.0,126.0,53.0
imap.,347.583333,6.299260e+02,0.0,1492.0,12.0
ipsweep.,10.083400,5.231658e+00,0.0,18.0,1247.0
land.,0.000000,0.000000e+00,0.0,0.0,21.0
loadmodule.,151.888889,1.277453e+02,0.0,302.0,9.0
multihop.,435.142857,5.409604e+02,0.0,1412.0,7.0
neptune.,0.000000,0.000000e+00,0.0,0.0,107201.0


And so on. By reusing the `summary_by_label` and `get_variable_stats_df` functions we can perform some exploratory data analysis in large datasets with Spark.  

### Correlations

Spark's MLlib supports [Pearson’s](http://en.wikipedia.org/wiki/Pearson_product-moment_correlation_coefficient) and [Spearman’s](http://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient) to calculate pairwise correlation methods among many series. Both of them are provided by the `corr` method in the `Statistics` package.  

We have two options as input. Either two `RDD[Double]`s or an `RDD[Vector]`. In the first case the output will be a `Double` value, while in the second a whole correlation Matrix. Due to the nature of our data, we will obtain the second.    

In [41]:
from pyspark.mllib.stat import Statistics 
correlation_matrix = Statistics.corr(vector_data, method="spearman")

Once we have the correlations ready, we can start inspecting their values.

In [42]:
import pandas as pd
pd.set_option('display.max_columns', 50)

col_names = ["duration","src_bytes","dst_bytes","land","wrong_fragment",
             "urgent","hot","num_failed_logins","logged_in","num_compromised",
             "root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds",
             "is_hot_login","is_guest_login","count","srv_count","serror_rate",
             "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

corr_df = pd.DataFrame(correlation_matrix, index=col_names, columns=col_names)

corr_df

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_hot_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
duration,1.000000,0.014196,0.299189,-0.001068,-0.008025,0.017883,0.108639,0.014363,0.159564,0.010687,0.040425,0.026015,0.013401,0.061099,0.008632,0.019407,-0.000020,-0.000010,0.205606,-0.259032,-0.250139,-0.074211,-0.073663,-0.025936,-0.026420,0.062291,-0.050875,0.123621,-0.161107,-0.217167,-0.211979,0.231644,-0.065202,0.100692,-0.056753,-0.057298,-0.007759,-0.013891
src_bytes,0.014196,1.000000,-0.167931,-0.009404,-0.019358,0.000094,0.113920,-0.008396,-0.089702,0.118562,0.003067,0.002282,-0.002050,0.027710,0.014403,-0.001497,0.000010,0.000019,0.027511,0.666230,0.722609,-0.657460,-0.652391,-0.342180,-0.332977,0.744046,-0.739988,-0.104042,0.130377,0.741979,0.729151,-0.712965,0.815039,-0.140231,-0.645920,-0.641792,-0.297338,-0.300581
dst_bytes,0.299189,-0.167931,1.000000,-0.003040,-0.022659,0.007234,0.193156,0.021952,0.882185,0.169772,0.026054,0.012192,-0.003884,0.034154,-0.000054,0.065776,-0.000030,0.000041,0.085947,-0.639157,-0.497683,-0.205848,-0.198715,-0.100958,-0.081307,0.229677,-0.222572,0.521003,-0.611972,0.024124,0.055033,-0.035073,-0.396195,0.578557,-0.167047,-0.158378,-0.003042,0.001621
land,-0.001068,-0.009404,-0.003040,1.000000,-0.000333,-0.000065,-0.000539,-0.000076,-0.002785,-0.000447,-0.000093,-0.000049,-0.000230,-0.000149,-0.000076,-0.000211,-0.002868,0.002099,-0.000250,-0.010939,-0.010128,0.014160,0.014342,-0.000451,-0.001690,0.002153,-0.001846,0.020678,-0.019923,-0.012341,0.002576,-0.001803,0.004265,0.016171,0.013566,0.012265,0.000389,-0.001816
wrong_fragment,-0.008025,-0.019358,-0.022659,-0.000333,1.000000,-0.000150,-0.004042,-0.000568,-0.020911,-0.003370,-0.000528,-0.000248,-0.001727,-0.001160,-0.000507,-0.001519,-0.000146,0.000442,-0.001869,-0.057711,-0.029117,-0.008849,-0.023382,0.000430,-0.012676,0.010218,-0.009386,0.012117,-0.029149,-0.058225,-0.049560,0.055542,-0.015449,0.007306,0.010387,-0.024117,0.046656,-0.013666
urgent,0.017883,0.000094,0.007234,-0.000065,-0.000150,1.000000,0.008594,0.063009,0.006821,0.031765,0.067437,0.000020,0.061994,0.061383,-0.000065,0.023380,0.012915,0.005191,-0.000100,-0.004778,-0.004799,-0.001338,-0.001327,-0.000705,-0.000726,0.001521,-0.001522,-0.000788,-0.005894,-0.005698,-0.004078,0.005208,-0.001939,-0.000976,-0.001381,-0.001370,-0.000786,-0.000782
hot,0.108639,0.113920,0.193156,-0.000539,-0.004042,0.008594,1.000000,0.112560,0.189126,0.811529,0.101983,-0.000400,0.003096,0.028694,0.009146,0.004224,-0.000392,-0.000247,0.463706,-0.120847,-0.114735,-0.035487,-0.034934,0.013468,0.052003,0.041342,-0.040555,0.032141,-0.074178,-0.017960,0.018783,-0.017198,-0.086998,-0.014141,-0.004706,-0.010721,0.199019,0.189142
num_failed_logins,0.014363,-0.008396,0.021952,-0.000076,-0.000568,0.063009,0.112560,1.000000,-0.002190,0.004619,0.016895,0.072748,0.010060,0.015211,-0.000093,0.005581,0.003439,-0.001554,-0.000428,-0.018024,-0.018027,-0.003674,-0.004027,0.035324,0.034877,0.005716,-0.005538,-0.003096,-0.028369,-0.015092,0.003004,-0.002960,-0.006617,-0.002588,0.014713,0.014914,0.032395,0.032151
logged_in,0.159564,-0.089702,0.882185,-0.002785,-0.020911,0.006821,0.189126,-0.002190,1.000000,0.161190,0.025293,0.011813,0.082533,0.055530,0.024354,0.072698,0.000079,0.000127,0.089318,-0.578287,-0.438947,-0.187114,-0.180122,-0.091962,-0.072287,0.216969,-0.214019,0.503807,-0.682721,0.080352,0.114526,-0.093565,-0.359506,0.659078,-0.143283,-0.132474,0.007236,0.012979
num_compromised,0.010687,0.118562,0.169772,-0.000447,-0.003370,0.031765,

We have used a Pandas DataFrame here to render the correlation matrix in a more comprehensive way. Now we want those variables that are highly correlated. For that we do a bit of dataframe manipulation.

In [43]:
# get a boolean dataframe where true means that a pair of variables is highly correlated
highly_correlated_df = (abs(corr_df) > .8) & (corr_df < 1.0)  # exclude self-correlation

# get the names of the variables so we can use them to slice the dataframe
correlated_vars_index = (highly_correlated_df==True).any()
correlated_var_names = correlated_vars_index[correlated_vars_index==True].index

# slice it
highly_correlated_df.loc[correlated_var_names,correlated_var_names]

,src_bytes,dst_bytes,hot,logged_in,num_compromised,num_outbound_cmds,is_hot_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
src_bytes,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
dst_bytes,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
hot,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
logged_in,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
num_compromised,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
num_outbound_cmds,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
is_hot_login,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
count,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
srv_count,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
serror_rate,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,True,True,False,False


#### Conclusions and possible model selection hints

The previous dataframe showed us which variables are highly correlated. We have kept just those variables with at least one strong correlation. We can use as we please, but a good way could be to do some model selection. That is, if we have a group of variables that are highly correlated, we can keep just one of them to represent the group under the assumption that they convey similar information as predictors. Reducing the number of variables will not improve our model accuracy, but it will make it easier to understand and also more efficient to compute.  

For example, from the description of the [KDD Cup 99 task](http://kdd.ics.uci.edu/databases/kddcup99/task.html) we know that the variable `dst_host_same_src_port_rate` references the percentage of the last 100 connections to the same port, for the same destination host. In our correlation matrix (and auxiliar dataframes) we find that this one is highly and positively correlated to `src_bytes` and `srv_count`. The former is the number of bytes sent form source to destination. The later is the number of connections to the same service as the current connection in the past 2 seconds. We might decide not to include `dst_host_same_src_port_rate` in our model if we include the other two, as a way to reduce the number of variables and later one better interpret our models.  

Later on, in those notebooks dedicated to build predictive models, we will make use of this information to build more interpretable models.   

The next 2 sections cover 

## MLlib: Classification with Logistic Regression  

In this notebook we will use Spark's machine learning library [MLlib](https://spark.apache.org/docs/latest/mllib-guide.html) to build a **Logistic Regression** classifier for network attack detection. We will use the complete [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) datasets in order to test Spark capabilities with large datasets.  

Additionally, we will introduce two ways of performing **model selection**: by using a correlation matrix and by using hypothesis testing.    

## MLlib: Decision Trees  

In this notebook we will use Spark's machine learning library [MLlib](https://spark.apache.org/docs/latest/mllib-guide.html) to build a **Decision Tree** classifier for network attack detection. We will use the complete [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) datasets in order to test Spark capabilities with large datasets. 

Decision trees are a popular machine learning tool in part because they are easy to interpret, handle categorical features, extend to the multiclass classification setting, do not require feature scaling, and are able to capture non-linearities and feature interactions. In this notebook, we will first train a classification tree including every single predictor. Then we will use our results to perform model selection. Once we find out the most important ones (the main splits in the tree) we will build a minimal tree using just three of them (the first two levels of the tree in order to compare performance and accuracy.   

At the time of processing this notebook, our Spark cluster contains:  

- Eight nodes, with one of them acting as master and the rest as workers.  
- Each node contains 8Gb of RAM, with 6Gb being used for each node.  
- Each node has a 2.4Ghz Intel dual core processor.  
- Running Apache Spark 1.3.1.  

Please refer to github for corresponding notebooks:

[Introduction to Spark with Python, by Jose A. Dianes](https://github.com/jadianes/spark-py-notebooks)